<a href="https://colab.research.google.com/github/shreya29k/Youtube-video-summerizer/blob/main/summarizeproject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install youtube-transcript-api
from youtube_transcript_api import YouTubeTranscriptApi
!pip install transformers

!pip install nltk
from nltk.tokenize import sent_tokenize
import nltk # Import the nltk module
nltk.download('punkt')

!pip install scikit-learn
from sklearn.feature_extraction.text import TfidfVectorizer # Import TfidfVectorizer


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
sub = YouTubeTranscriptApi.get_transcript("i6AZdFxTK9I")


In [ ]:
subtitle = " ".join([x['text'] for x in sub])


subtitle = subtitle.replace("\n", " ")
sentences = sent_tokenize(subtitle)





In [ ]:
tf_idf = TfidfVectorizer(min_df=1,
                         strip_accents='unicode',
                         max_features=None,
                         lowercase=True,
                         token_pattern=r'\w{1,}',
                         ngram_range=(1, 3),
                         use_idf=1,
                         smooth_idf=1,
                         sublinear_tf=1,
                         stop_words='english')

import numpy as np

sentence_vectors = tf_idf.fit_transform(sentences)
sent_scores = np.array(sentence_vectors.sum(axis=1)).ravel()

N = 3
top_n_sentences = [sentences[index] for index in np.argsort(sent_scores, axis=0)[::-1][:N]]

# mapping the scored sentences with their indexes as in the subtitle
organized_sent = {k:v for v,k in enumerate(sentences)}
mapped_sentences = [(sentence, organized_sent[sentence]) for sentence in top_n_sentences]
mapped_sentences = sorted(mapped_sentences, key=lambda x: x[1])
ordered_sentences = [element[0] for element in mapped_sentences]
summary = " ".join(ordered_sentences)


In [ ]:
from transformers import BartTokenizer, BartForConditionalGeneration

tokenizer = BartTokenizer.from_pretrained('facebook/bart-large-cnn')
model = BartForConditionalGeneration.from_pretrained('facebook/bart-large-cnn')

input_tensor = tokenizer.encode(subtitle, return_tensors="pt", max_length=512)

outputs_tensor = model.generate(input_tensor, max_length=160, min_length=120, length_penalty=2.0, num_beams=4, early_stopping=True)
print(tokenizer.decode(outputs_tensor[0]))

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


</s><s>With over 3 billion devices running java it has helped shape the world as we know it today. The biggest internet company google has a huge percentage of its development in java accenture is using java for most of its clients hardware companies like intel symantec philips thompson are all java users. java is also considered the official language for android game development pubg the latest youth obsession is a cross-platform game that uses java as a sourcing language since 2007 java has remained a de facto standard and is controlled through the java community process java has evolved to become the most popularly used language.</s>


In [ ]:
from transformers import pipeline

summarizer = pipeline('summarization')

# Truncate the input to a maximum length that the model can handle
max_input_length = 1024  # Adjust this value based on the model's limitations
truncated_subtitle = subtitle[:max_input_length]

summary = summarizer(truncated_subtitle, max_length=180, min_length=30)
print(summary)

No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 and revision a4f8f3e (https://huggingface.co/sshleifer/distilbart-cnn-12-6).
Using a pipeline without specifying a model name and revision in production is not recommended.


[{'summary_text': ' The first public implementation of java was java 1.0 in 1995 . Over 3 billion devices are now running java . The language has helped shape the world as we know it today .'}]
